In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

import session
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)

objp *= 3
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
sessionPath = "/Volumes/Public/Shared Files/KamerJelle/session-2022-03-04 11-41-13"
calibrationSession = session.Session().from_path(sessionPath)

for image in calibrationSession.imageTransforms:
    print("Calibrating", image.id)
    gray = cv2.cvtColor(cv2.imread(image.path), cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 7), None)

    #plt.imshow(gray)
    #plt.show()

    # If found, add object points, image points (after refining them)
    if ret:
        print("found Chessboard")
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

    # Calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print(ret,"\n", mtx,"\n", dist,"\n", rvecs,"\n", tvecs)

In [ ]:
calibrationSession.imageTransforms[0].fov *=2
print(calibrationSession.imageTransforms[0].get_camera_matrix())
